# Starfish SeqFISH Work-in-progress Processing Example

In [ ]:
%gui qt

import os
from copy import deepcopy
from itertools import product

import numpy as np
import pandas as pd
import skimage.filters
import skimage.morphology
from skimage.transform import SimilarityTransform, warp
from tqdm import tqdm

import starfish
import starfish.data
from starfish.types import Axes, Levels, TraceBuildingStrategies

Select data for a single field of view.

In [ ]:
exp = starfish.data.SeqFISH(use_test_data=True)

In [ ]:
img = exp['fov_000'].get_image('primary')

The first step in SeqFISH is to do some rough registration. For this data, the rough registration has been done for us by the authors, so it is omitted from this notebook.

## Remove image background

To remove image background, use a White Tophat filter, which measures the background with a rolling disk morphological element and subtracts it from the image.

In [ ]:
from skimage.morphology import opening, dilation, disk
from functools import partial

If desired, the background that is being subtracted can be visualized

In [ ]:
opening = partial(opening, selem=disk(3))

background = img.apply(
    opening,
    group_by={Axes.ROUND, Axes.CH, Axes.ZPLANE}, verbose=False, in_place=False
)

starfish.display(background)

In [ ]:
wth = starfish.image.Filter.WhiteTophat(masking_radius=3)
background_corrected = wth.run(img, in_place=False)
starfish.display(background_corrected)

## Scale images to equalize spot intensities across channels

The number of peaks are not uniform across rounds and channels, which prevents histogram matching across channels. Instead, a percentile value is identified and set as the maximum across channels, and the dynamic range is extended to equalize the channel intensities

In [ ]:
clip = starfish.image.Filter.Clip(p_max=99.9, is_volume=True, level_method=Levels.SCALE_BY_CHUNK)
scaled = clip.run(background_corrected, in_place=False)

In [ ]:
starfish.display(scaled)

## Remove residual background

The background is fairly uniformly present below intensity=0.5. However, starfish's clip method currently only supports percentiles. To solve this problem, the intensities can be directly edited in the underlying numpy array.

In [ ]:
from copy import deepcopy
clipped = deepcopy(scaled)
clipped.xarray.values[clipped.xarray.values < 0.7] = 0

In [ ]:
starfish.display(clipped)

## Detect Spots

Detect spots with a local search blob detector that identifies spots in all rounds and channels and matches them using a local search method. The local search starts in an anchor channel (default ch=1) and identifies the nearest spot in all subsequent imaging rounds.

In [ ]:
threshold = 0.5

bd = starfish.spots.FindSpots.BlobDetector(
    min_sigma=(1.5, 1.5, 1.5),
    max_sigma=(8, 8, 8),
    num_sigma=10,
    threshold=threshold)

spots = bd.run(clipped)
decoder = starfish.spots.DecodeSpots.PerRoundMaxChannel(
    codebook=exp.codebook,
    search_radius=7,
    trace_building_strategy=TraceBuildingStrategies.NEAREST_NEIGHBOR)

decoded = decoder.run(spots=spots)

In [ ]:
starfish.display(clipped, decoded)

Based on visual inspection, it looks like the spot correspondence across rounds isn't being detected well. Try the PixelSpotDecoder.

In [ ]:
glp = starfish.image.Filter.GaussianLowPass(sigma=(0.3, 1, 1), is_volume=True)
blurred = glp.run(clipped)

In [ ]:
psd = starfish.spots.DetectPixels.PixelSpotDecoder(
    codebook=exp.codebook, metric='euclidean', distance_threshold=0.5,
    magnitude_threshold=0.1, min_area=7, max_area=50,
)
pixel_decoded, ccdr = psd.run(blurred)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# look at the label image in napari
label_image = starfish.ImageStack.from_numpy(np.reshape(ccdr.decoded_image, (1, 1, 29, 280, 280)))
starfish.display(label_image)

Compare the number of spots being detected by the two spot finders

In [ ]:
print("pixel_decoder spots detected", int(np.sum(pixel_decoded['target'] != 'nan')))
print("local search spot detector spots detected", int(np.sum(decoded['target'] != 'nan')))

Report the correlation between the two methods

In [ ]:
from scipy.stats import pearsonr

# get the total counts for each gene from each spot detector
pixel_decoded_gene_counts = pd.Series(*np.unique(pixel_decoded['target'], return_counts=True)[::-1])
decoded_gene_counts = pd.Series(*np.unique(decoded['target'], return_counts=True)[::-1])

# get the genes that are detected by both spot finders
codetected = pixel_decoded_gene_counts.index.intersection(decoded_gene_counts.index).drop('nan')

# report the correlation
pearsonr(pixel_decoded_gene_counts[codetected], decoded_gene_counts[codetected])